加载config example

In [1]:
import json
import os

from loguru import logger
import pandas as pd

'''
训练和推理的datasource不同，因此
测试训练的config，在下方选择config-train-example.json
测试推理的config，在下方选择config-inference-example.json
'''

config_file_path = "../config/example/config-train-example.json"
# config_file_path = "../config/example/config-inference-example.json"
logger.info("config file is : {}".format(config_file_path))

with open(config_file_path) as config_file:
    config_content = config_file.read()

json_config = json.loads(config_content)
print(json_config)

# 为config设置环境变量
os.environ["MODE"] = "TRAIN"
os.environ["CONFIG"] = config_content

2023-02-24 22:55:18.509 | INFO     | __main__:<cell line: 15>:15 - config file is : ../config/example/config-train-example.json


{'hyperParams': {'batch_size': 32, 'epochs': 10}, 'modelPath': 'testModelPath', 'resultPath': 'result', 'trainJobName': 'nginx-pure-data-ecl-64778820', 'datasetStartTime': -1, 'ckptName': 'nginx-pure_data.ECL-64778820', 'storage': [{'type': 'REDIS', 'usage': 'model', 'host': 'localhost', 'port': 6379, 'password': '', 'properties': {'db': 0}}, {'type': 'MONGODB', 'usage': 'result', 'host': '10.112.169.94', 'port': 30332, 'username': 'root', 'password': 'root', 'properties': {'db': 'evaluation'}}], 'selectedFields': ['MT_000', 'MT_001', 'date'], 'algorithm': {'type': 'KNN', 'name': 'nginx', 'description': 'nginx', 'author': 'wanz', 'version': '1.0', 'url': 'nginx', 'hyperParameters': {'hello': 'world'}, 'command': ' ', 'hashString': '104760218104760218364187848563104760218197974952332', 'id': '63f320a11a1cf03bbd8f01c6'}, 'datasetEndTime': -1, 'dataSource': {'type': 'MONGODB', 'name': 'pure_data.ECL', 'host': '10.112.169.94', 'port': 30332, 'username': 'root', 'password': 'root', 'propert

测试config基本字段

In [2]:
from sdk.config.config import Config

config = Config()
print("selected fields: {}".format(config.selected_fields))
print("algorithm name: {}".format(config.algorithm.get_name()))

selected fields: ['MT_000', 'MT_001', 'date']
algorithm name: nginx


测试storage

In [3]:
# model storage and redis storage
model_storage = config.get_model_storage()


class TestModel:
    def __init__(self):
        self.data = 1

    def get_data(self):
        return self.data


model = TestModel()

model_storage.save_model(config.model_path, model, {"min": 1, "max": 2})
model_retrieve, args = model_storage.load_model(config.model_path)
print(model_retrieve.get_data())
print(args)

# result storage and mongo storage
result_storage = config.get_result_storage()
result_storage.store_result(config.result_path, "test-algo", ["test-metric1"], ["test-target1"], pd.Series(["1"]))

1
{'min': 1, 'max': 2}


测试datasource

In [4]:
# train datasource and mongo datasource
if "train" in config_file_path:
    train_datasource = config.get_train_datasource()
    train_datasource.set_default_projected_fields({"_id": 0})
    train_datasource.set_default_df_index("date")
    data = train_datasource.get_data(config.dataset_start_time, config.dataset_end_time, config.selected_fields)

# inference datasource and kafka datasource
elif "inference" in config_file_path:
    inference_datasource = config.get_inference_datasource()
    stream = inference_datasource.get_stream_handler()

    count = 0
    for message in stream:
        mvalue = message.value
        print(mvalue)
        count += 1
        if count == 5:
            break

2023-02-24 22:55:19.227 | INFO     | sdk.config.datasource.mongo_datasource:get_data:47 - get train data dataframe, head is :                     MT_000 MT_001
date                             
2012-01-01 00:00:00   14.0   69.0
2012-01-01 01:00:00   18.0   92.0
2012-01-01 02:00:00   21.0   96.0
2012-01-01 03:00:00   20.0   92.0
2012-01-01 04:00:00   22.0   91.0
